<div class="alert alert-block alert-success">
    This notebook is intended to act as a template for the example notebooks that use the raster API. These green cells should all be deleted and in several sections only one of the provided cells should be included in the notebook.
</div>

<div class="alert alert-block alert-success">

Update the link in the following section.
    
</div>

## Run this notebook

You can launch this notbook using mybinder, by clicking the button below.

<a href="https://binder.openveda.cloud/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=user-guide/notebooks/templates/template-using-the-raster-api.ipynb">
<img src="https://binder.openveda.cloud/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

<div class="alert alert-block alert-success">
    
Fill in the text in _italics_ in the following cells

</div>

## Approach

   1. _list a few steps that outline the approach_
   2. _you will be taking in this notebook_

In [ ]:
# include all your imports in this cell
import folium
import requests

## About the data

_Optional description of the dataset._

## Declare your collection of interest

You can discover available collections the following ways:

* Programmatically: see example in the `list-collections.ipynb` notebook
* JSON API: https://openveda.cloud/api/stac/collections
* STAC Browser: http://openveda.cloud

In [ ]:
STAC_API_URL = "https://openveda.cloud/api/stac"
RASTER_API_URL = "https://openveda.cloud/api/raster"

collection_id = 

<div class="alert alert-block alert-success">
    
Next step is to get STAC objects from the STAC API. In some notebooks we get the collection and use all the items, and in others we search for specific items.

</div>

## Fetch STAC collection

We will use `requests` to fetch all the metadata about the collection of interest from STAC.

In [ ]:
collection = requests.get(f"{STAC_API_URL}/collections/{collection_id}").json()
collection

## Fetch STAC item for a particular time

We can use the search API to find the item that matches exactly our time of interest.

In [ ]:
response = requests.post(
    f"{STAC_API_URL}/search",
    json={
        "collections": [collection_id],
        "query": {"datetime": {"eq": "2021-01-01T00:00:00"}},
        "limit": 100,
    },
).json()
items = response["features"]

<div class="alert alert-block alert-success">
The next step is often to define an Area of Interest. Note that it is preferred to get large geojson objects directly from their source rather than storing them in this repository or inlining them in the notebook. Here is an example of what that might look like. 
</div>

## Define an AOI

We can fetch GeoJSON for metropolitan France and Corsica (excluding overseas territories) from an authoritative online source (https://gadm.org/download_country.html).

In [ ]:
response = requests.get(
    "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_FRA_0.json"
)

# If anything goes wrong with this request output error contents
assert response.ok, response.text

result = response.json()
print(f"There are {len(result['features'])} features in this collection")

That is the geojson for a feature collection, but since there is only one feature in it we can grab just that.

In [ ]:
aoi = result["features"][0]

Let's take a look at this AOI on a map

In [ ]:
m = folium.Map(
    location=[40, 0],
    zoom_start=2,
)

folium.GeoJson(aoi, name="AOI").add_to(m)
m

<div class="alert alert-block alert-success">
    
With the STAC object and optionally the AOI in hand, the next step is to do some analysis. The sections in the rest of the notebooks are totally up to you! Here is one idea though :)
    
</div>

## Use the STAC Item to get tiles with the RASTER API

We pass the item_id, collection id, and `rescale_values` in to the RASTER API `/collections/{collection_id}/items/{item_id}/tilejson.json` endpoint and get back a tile. See the tips below for choosing visualization parameters for your tiles.

<div class="alert alert-block alert-info">
<b>Colormap Tip:</b>
    <br/>Find the list of available colormaps at <code>{RASTER_API}/colorMaps</code> 
    (<a href=https://openveda.cloud/api/raster/colorMaps>openveda.cloud/api/raster/colorMaps</a>) 
    and get colormap metadata and/or legend image at 
<code>{RASTER_API}/colorMaps/{colorMapName}</code> (See docs at <a href=https://openveda.cloud/api/raster/docs#/ColorMaps/getColorMap>openveda.cloud/api/raster/docs#/ColorMaps/getColorMap</a>)
</div>


<div class="alert alert-block alert-info">
<b>Tiling schemes Tip:</b>
    <br/>Find the list of available tile matrix set ids at <code>{RASTER_API}/tileMatrixSets</code> 
(<a href=https://openveda.cloud/api/raster/tileMatrixSets>openveda.cloud/api/raster/tileMatrixSets</a>) and get tiling scheme metadata at <code>{RASTER_API}/colorMaps/{colorMapName}</code> (See docs at <a href=https://openveda.cloud/api/raster/docs#/tileMatrixSets/tileMatrixSetId>openveda.cloud/api/raster/docs#/tileMatrixSets/tileMatrixSetId</a>)
</div>

<div class="alert alert-block alert-info">
<b>Raster rescale range tip:</b>
    <br/>Get the statistics for item assets at <code>{RASTER_API}/collections/{collection_id/items/{item_id}/statistics</code> 
    (<a href=https://openveda.cloud/api/raster/docs#/STAC%20Item/statistics_collections__collection_id__items__item_id__statistics_get</a>)
</div>


In [ ]:
# Start with the first item returned
item = items[0]

# Here is a default tile matrix id
tile_matrix_set_id = "WebMercatorQuad"

# Adjust these values to find the ideal range for the range of data you will be visualizing
rescale_min = 0
rescale_max = 1

# Set the asset key you want to visualize (many STAC Items in the VEDA catalog have a cog_default assets)
asset_key = "cog_default"

# Choose a colormap
colormap_name = "viridis"

# Use stac item url with to get tilejson from raster api
url = f"{RASTER_API_URL}/collections/{collection_id}/items/{item['id']}/{tile_matrix_set_id}/tilejson.json"

tiles = requests.get(
    url,
    params = {
        "assets": asset_key,
        "colormap_name": colormap_name,
        "rescale": f"{rescale_min},{rescale_max}"
    }
).json()
tiles

With that tile url in hand we can create a simple visualization using `folium`.

In [ ]:
folium.Map(
    tiles=tiles["tiles"][0],
    attr="VEDA",
)